In [4]:
import numpy as np

#### Read data from file

In [5]:
with open ("data.txt", 'r') as data: #read data 
    dataset = [[int(x) for x in line.split()] for line in data]  

In [6]:
make_R = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
R = make_R(dataset, 15)
print("Number of R: {}".format(len(R)))

Number of R: 10


#### K-optimization

In [7]:
def check_x(S):
    max_k = np.array([])
    opt_k = np.array([])
    for i, x in zip(range(len(S)),S):
        if np.any(S-x > 0): #формуємо множину k-макс. елементів
            continue
        max_k = np.append(max_k, i+1)
        if np.sum(x) == 15: #формуємо множину k-опт. елементів
            opt_k = np.append(opt_k, i+1)
        
    return max_k, opt_k

In [8]:
def k_optimization(Rn):
    I = (Rn == Rn.T)*Rn #формуємо симетричну частину
    P = Rn-I #формуємо асиметричну частину
    N = (Rn == Rn.T)-I #формуємо відношення непорівнюваності
    
    for i in range(1,5):
        if i==1:
            S1=I+P+N
            max_1, opt_1 = check_x(S1)
        elif i==2:
            S2=P+N
            max_2, opt_2 = check_x(S2)
        elif i==3:
            S3=P+I
            max_3, opt_3 = check_x(S3)
        elif i==4:
            S4=P
            max_4, opt_4 = check_x(S4)
            
    parameters = {"1_max": max_1,
                 "1_opt": opt_1,
                 "2_max": max_2,
                 "2_opt": opt_2,
                 "3_max": max_3,
                 "3_opt": opt_3,
                 "4_max": max_4,
                 "4_opt": opt_4}
    
    return parameters

In [18]:
def print_information_k_opt(parameters, num_r):
    print("____R{}____".format(num_r))
    print("1-max: {}".format(parameters["1_max"]))
    print("1-opt: {}".format(parameters["1_opt"]))
    print("2-max: {}".format(parameters["2_max"]))
    print("2-opt: {}".format(parameters["2_opt"]))
    print("3-max: {}".format(parameters["3_max"]))
    print("3-opt: {}".format(parameters["3_opt"]))
    print("4-max: {}".format(parameters["4_max"]))
    print("4-opt: {}".format(parameters["4_opt"])) 

In [19]:
for num_r, r_i in zip(range(len(R)),R):
    Rn = np.array(r_i)
    parameters = k_optimization(Rn)
    print_information_k_opt(parameters, num_r+1)

____R1____
1-max: [3.]
1-opt: [3.]
2-max: [3.]
2-opt: []
3-max: [3.]
3-opt: [3.]
4-max: [3.]
4-opt: []
____R2____
1-max: [3.]
1-opt: [3.]
2-max: [3.]
2-opt: [3.]
3-max: []
3-opt: []
4-max: []
4-opt: []
____R3____
1-max: [2.]
1-opt: [2.]
2-max: [2.]
2-opt: [2.]
3-max: []
3-opt: []
4-max: []
4-opt: []
____R4____
1-max: []
1-opt: []
2-max: []
2-opt: []
3-max: []
3-opt: []
4-max: []
4-opt: []
____R5____
1-max: [1.]
1-opt: [1.]
2-max: [1.]
2-opt: [1.]
3-max: []
3-opt: []
4-max: []
4-opt: []
____R6____
1-max: [ 5.  6.  8. 11. 14.]
1-opt: [ 5.  6.  8. 11. 14.]
2-max: [ 5.  6. 11.]
2-opt: [ 5.  6. 11.]
3-max: []
3-opt: []
4-max: [ 5.  6. 11.]
4-opt: []
____R7____
1-max: [ 4.  7.  8. 13. 14.]
1-opt: [ 4.  7.  8. 13. 14.]
2-max: [ 4.  8. 13.]
2-opt: [ 4.  8. 13.]
3-max: []
3-opt: []
4-max: [ 4.  8. 13.]
4-opt: []
____R8____
1-max: []
1-opt: []
2-max: []
2-opt: []
3-max: []
3-opt: []
4-max: []
4-opt: []
____R9____
1-max: [ 1.  4.  7.  9. 10. 11.]
1-opt: [ 1.  4.  7.  9. 10. 11.]
2-max: [ 1.  4.  

#### Neumann-Morgenstern optimization

In [68]:
def Neumann_Morgenstern_optimization(Rn):
    print(Rn)
    
    check_S0 = np.all(Rn == 0, axis=0, keepdims=True)
    
    print("check_s0:  ", check_S0)
    
    index_S0 = np.where(check_S0[0]==True)[0]
    
    print("index_S0   ", index_S0)
    
    S_candidate = {"S0": index_S0}
    S = {"S0": index_S0}
    
    index_curr_S = 0
    
    print("______good__________________________________________________________")
    
    while(len(S["S"+str(index_curr_S)]) <= len(Rn)):#1 step 
        

        print("---------current S index--------------")
        print("S"+str(index_curr_S), "     ", S["S"+str(index_curr_S)])
        
        index_col = np.array([], dtype=int)
        for index in S["S"+str(index_curr_S)]:
            index_col_with_true = np.where(Rn[index]==True)#індекси стовпців елеентів, що мають 1 на пертині з рядком
            index_col = np.append(index_col, index_col_with_true) #массив для зберігання унікальних значень всіх індексів
        index_col = np.unique(index_col)
        print("inDEX COL", index_col)
    
    
        S_next = np.array([], dtype=int) 
        
        for index in index_col:#перевіряємо кожен стовпець. що відповідає елементу. індекс якого ми запам'ятали
            index_col_elements = np.where(Rn.T[index]==True) #індекси одиниць в колонках для подальшої перевірки на їх включення до S
#             print(index_col_elements)#для кожної колонки де знах одиниці пам'ятаємо
            inclusion_check = np.in1d(index_col_elements, S["S"+str(index_curr_S)]) #включається ли колонка тольлько в строки из S 
#             print("INCLUSION CHECK", inclusion_check)
            
            
            if np.all(inclusion_check == True):
                S_next = np.append(S_next, index) # альтернативи які будемо додавати. різниця між Sk -Sk-1
                
        print("s_next  добавление ", S_next)
        
        index_curr_S += 1

        S_candidate["S"+str(index_curr_S)] = S_next
        S["S"+str(index_curr_S)] = np.append(S["S"+str(index_curr_S)], S_next)
        
        print(S)
        print(S_candidate)
        
#         print(len(S["S"+str(index_curr_S)]))
#         print(len(Rn))
        
#     print(S["S"+str(index_curr_S)])
    
#         print(index_col_elements) 
#         print("__________")
            







In [69]:
Neumann_Morgenstern_optimization(np.array(R[5]))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 0 1]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 0 1]
 [0 0 0 0 0 0 1 0 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 1 0 1 1 0 1 1 0 0]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 0 1]
 [0 0 0 0 0 0 1 0 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 1 0 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
check_s0:   [[False False False False  True  True False False False False  True False
  False False False]]
index_S0    [ 4  5 10]
______good__________________________________________________________
---------current S index--------------
S0       [ 4  5 10]
inDEX COL [ 0  1  2  3  6  8  9 11 12 14]
s_next  добавление  [ 0  2 14]


KeyError: 'S1'

#### Check acyclicity

In [20]:
def acyclicity(matrix): 
    size = len(matrix)
    oppos_matrix = opposite(matrix)
    current_matrix = matrix[:]
    if intersection(matrix, oppos_matrix).any():
        return False
    compos_matrix=composition(matrix,matrix)
    while((compos_matrix == current_matrix).all()):
        matrix = compos_matrix[:]
        compos_matrix = composition(compos_matrix,matrix)
        if intersection(matrix,oppos_matrix).any():
            return False
    return True

def intersection(matrix1, matrix2):
    size = len(matrix1)
    intersection_matrix = np.empty((size,size))
    for i in range(size):
        for j in range(size):
            intersection_matrix[i][j] = matrix1[i][j] and matrix2[i][j]
    return intersection_matrix

def composition(matrix1, matrix2): 
    size = len(matrix1)
    compos_matrix = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            if matrix1[i][j] == 1:
                for k in range(size):
                    if matrix2[j][k] == 1:
                        compos_matrix[i][k] = 1
    return compos_matrix

def opposite(matrix): 
    return np.transpose(matrix)

#### Сomputation

In [22]:
for num_r, r_i in zip(range(len(R)),R):
    Rn = np.array(r_i)
    
    if acyclicity(Rn):
        pass
    else:
        parameters = k_optimization(Rn)
        print_information_k_opt(parameters, num_r+1)


____R1____
1-max: [3.]
1-opt: [3.]
2-max: [3.]
2-opt: []
3-max: [3.]
3-opt: [3.]
4-max: [3.]
4-opt: []
____R6____
1-max: [ 5.  6.  8. 11. 14.]
1-opt: [ 5.  6.  8. 11. 14.]
2-max: [ 5.  6. 11.]
2-opt: [ 5.  6. 11.]
3-max: []
3-opt: []
4-max: [ 5.  6. 11.]
4-opt: []
____R7____
1-max: [ 4.  7.  8. 13. 14.]
1-opt: [ 4.  7.  8. 13. 14.]
2-max: [ 4.  8. 13.]
2-opt: [ 4.  8. 13.]
3-max: []
3-opt: []
4-max: [ 4.  8. 13.]
4-opt: []
____R9____
1-max: [ 1.  4.  7.  9. 10. 11.]
1-opt: [ 1.  4.  7.  9. 10. 11.]
2-max: [ 1.  4.  7.  9. 10. 11.]
2-opt: [ 1.  4.  7.  9. 10. 11.]
3-max: [ 1.  3.  5.  7. 11.]
3-opt: []
4-max: [ 1.  7. 11.]
4-opt: []


In [48]:
np.array(R[5])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])